In [ ]:


# k, now we need to load up data slice

ds = yt.load("IsolatedGalaxy/galaxy0030/galaxy0030")


slc = ds.slice(1,.5)
frb = slc.to_frb(1., (100,100))
vals_fixed_y = frb[('gas','density')]


slc = ds.slice(0,.5)
frb = slc.to_frb(1., (100,100))
vals_fixed_x = frb[('gas','density')]


# 


def get_plane_from_frb(nx,ny,nz, frb_vals):


    x = np.linspace(0, 1, nx) # element-centers
    z = np.linspace(0, 1, nz) # element-centers
    y = np.linspace(0, 1, ny)

    def get_dval(xc,dmin = 0.01):
        if len(xc) > 1:
            return xc[1] - xc[0]
        else:
            return dmin

    def get_edge(xc, dx):
        if len(xc) > 1:
            return np.append([xc - dx/2.], [xc[-1] + dx/2])
        else:
            return np.array([0.5 - dx/2., 0.5 + dx/2.])

    dx = get_dval(x)
    dz = get_dval(z)
    dy = get_dval(y)

    xe = get_edge(x, dx)
    ye = get_edge(y, dy)
    ze = get_edge(z, dz)

    # assemble elements
    coords = []
    connectivity = []
    element_center_data = []
    icoord = 0

    for ix in range(nx):
        for iy in range(ny):
            for iz in range(nz):

                verts = [
                            [xe[ix], ye[iy], ze[iz]],
                            [xe[ix+1], ye[iy], ze[iz]],
                            [xe[ix+1], ye[iy+1], ze[iz]],
                            [xe[ix], ye[iy+1], ze[iz]],
                            [xe[ix], ye[iy], ze[iz+1]],
                            [xe[ix+1], ye[iy], ze[iz+1]],
                            [xe[ix+1], ye[iy+1], ze[iz+1]],
                            [xe[ix], ye[iy+1], ze[iz+1]]
                        ]

                connectivity += [[i for i in range(icoord, icoord+8)],]

                coords += verts
                icoord += 8

                if nx == 1:
                    these_vals = frb_vals[iy, iz]
                elif ny ==1:
                    these_vals = frb_vals[ix, iz]
                elif nz ==1:
                    these_vals = frb_vals[ix, iy]

                element_center_data.append(these_vals)

    coords = np.array(coords)
    connectivity = np.array(connectivity)


    return coords, connectivity, element_center_data

full_con = []
full_coord = []
full_data = []
offset = 0
for nx,ny,nz, vals in [(100,1, 100, vals_fixed_y), (1,100, 100, vals_fixed_x) ]:
    coord, conn, data = get_plane_from_frb(nx, ny, nz, vals)

    full_con.append(conn + offset)
    offset += coord.shape[0]
    full_data.append(data)
    full_coord.append(coord)

full_coord = np.concatenate(full_coord)
full_con = np.concatenate(full_con)
full_data = {('connect1','test'): np.concatenate(full_data)}




In [3]:
import numpy as np


/home/chris/src/yt_/yt_dev/yt/yt/utilities/logger.py:4: VisibleDeprecationWarning: The configuration file /home/chris/.config/yt/ytrc is deprecated in favor of /home/chris/.config/yt/yt.toml. Currently, both are present. Please manually remove the deprecated one to silence this warning.
Deprecated since v4.0.0 . This feature will be removed in v4.1.0
  from yt.config import ytcfg


In [36]:
import numpy as np

def assemble_brute(xe, ye, ze, nx, ny, nz, field, frb_cell_center_vals):
    # assemble elements! 
    print("assembling unstructured mesh for slice")    

    # containers that can be given to yt.load_unstructured()
    coords = []
    connectivity = []
    element_center_data = []
    icoord = 0

    # this could be vectorized... but it's not as bad as it could be since
    # one of the dimensions will always have 1 element... 
    for ix in range(nx):
        for iy in range(ny):
            for iz in range(nz):

                # add vertices of our hexahedral elements (in the proper order!)
                # to our coordinate array (this will repeat vertices in the 
                # coordinate array)
                verts = [
                            [xe[ix], ye[iy], ze[iz]],
                            [xe[ix+1], ye[iy], ze[iz]],
                            [xe[ix+1], ye[iy+1], ze[iz]],
                            [xe[ix], ye[iy+1], ze[iz]],
                            [xe[ix], ye[iy], ze[iz+1]],
                            [xe[ix+1], ye[iy], ze[iz+1]],
                            [xe[ix+1], ye[iy+1], ze[iz+1]],
                            [xe[ix], ye[iy+1], ze[iz+1]]
                        ]
                coords += verts
                
                # because we are repeating our verticies, connectivity can 
                # just be incremented
                connectivity += [[i for i in range(icoord, icoord+8)],]                
                icoord += 8

                # now extract our frb values for our cell_center values. 
                # the index order depends on which axis is orthoganal 
                # to our slice!
                if nx == 1:
                    these_vals = frb_cell_center_vals[iy, iz]
                elif ny ==1:
                    these_vals = frb_cell_center_vals[ix, iz]
                elif nz ==1:
                    these_vals = frb_cell_center_vals[ix, iy]
                else:
                    raise ValueError("One of the mesh dimensions must have a single element!")
                
                element_center_data.append(these_vals)

    # finalize our arrays which will now be ready to load as an unstructured_mesh! 
    coords = np.array(coords)
    connectivity = np.array(connectivity)
    
    if type(field) == tuple:
        field = field[1]
        
    element_center_data = {('connect1', field): np.array(element_center_data)}
    print("unstructured mesh assembled!")    
    return coords, connectivity, element_center_data


def assemble_smart(xe, ye, ze, nx, ny, nz, field, frb_cell_center_vals):
    # assemble elements! 
    print("assembling unstructured mesh for slice")    

    # containers that can be given to yt.load_unstructured()
    coords = []
    connectivity = []
    element_center_data = []
    icoord = 0

    # this could be vectorized... but it's not as bad as it could be since
    # one of the dimensions will always have 1 element... 
    ix, iy, iz = np.meshgrid(np.arange(0,nx), np.arange(0,ny), np.arange(0,nz))
    
    
    
    for ix in range(nx):
        for iy in range(ny):
            for iz in range(nz):

                # add vertices of our hexahedral elements (in the proper order!)
                # to our coordinate array (this will repeat vertices in the 
                # coordinate array)
                verts = [
                            [xe[ix], ye[iy], ze[iz]],
                            [xe[ix+1], ye[iy], ze[iz]],
                            [xe[ix+1], ye[iy+1], ze[iz]],
                            [xe[ix], ye[iy+1], ze[iz]],
                            [xe[ix], ye[iy], ze[iz+1]],
                            [xe[ix+1], ye[iy], ze[iz+1]],
                            [xe[ix+1], ye[iy+1], ze[iz+1]],
                            [xe[ix], ye[iy+1], ze[iz+1]]
                        ]
                coords += verts
                
                # because we are repeating our verticies, connectivity can 
                # just be incremented
                connectivity += [[i for i in range(icoord, icoord+8)],]                
                icoord += 8

                # now extract our frb values for our cell_center values. 
                # the index order depends on which axis is orthoganal 
                # to our slice!
                if nx == 1:
                    these_vals = frb_cell_center_vals[iy, iz]
                elif ny ==1:
                    these_vals = frb_cell_center_vals[ix, iz]
                elif nz ==1:
                    these_vals = frb_cell_center_vals[ix, iy]
                else:
                    raise ValueError("One of the mesh dimensions must have a single element!")
                
                element_center_data.append(these_vals)

    # finalize our arrays which will now be ready to load as an unstructured_mesh! 
    coords = np.array(coords)
    connectivity = np.array(connectivity)
    
    if type(field) == tuple:
        field = field[1]
        
    element_center_data = {('connect1', field): np.array(element_center_data)}
    print("unstructured mesh assembled!")    
    return coords, connectivity, element_center_data


def unstructured_ds_from_frb_slice(frb, field):

    # extract frb values and extract slice info
    frb_vals = frb[field] 
    parent_slice = frb.data_source 
    coord = parent_slice.coord 
    bounds = frb.bounds
    axis = parent_slice.axis 
    
    # build a quasi-3d mesh. frb values will be cell-centered values, will build a grid to 
    # to surround the cell-centered values.
    
    # first decide on the thickness of our cells in the slice-orthoganal direction
    d1 = (bounds[1] - bounds[0]) / frb.buff_size[0]
    d2 = (bounds[2] - bounds[3]) / frb.buff_size[1]
    pseudo_d = np.min([d1, d2]) / 1e2 # our cell width in the slice-orthoganal axis
    pseudo_bounds = [coord - pseudo_d/2., coord + pseudo_d/2.]
    
    # now set the bounds and number of cells in each direction. The axis orthogonal to slicing plane will 
    # always have 1 element. 
    if axis == 0: 
        nx = 1
        xlims = pseudo_bounds
        ny = frb.buff_size[0]
        ylims = [bounds[0], bounds[1]]
        nz = frb.buff_size[1]
        zlims = [bounds[2], bounds[3]]
    elif axis == 1: 
        nx = frb.buff_size[0]
        xlims = [bounds[0], bounds[1]]
        ny = 1
        ylims = pseudo_bounds
        nz = frb.buff_size[1]  
        zlims = [bounds[2], bounds[3]]
    elif axis == 2: 
        nx = frb.buff_size[0]
        xlims = [bounds[0], bounds[1]]
        ny = frb.buff_size[1] 
        ylims = [bounds[2], bounds[3]]
        nz = 1
        ylims = pseudo_bounds
        
    
    # construct cell-center arrays, cell widths and cell vertices arrays for each axis
    def get_grid_axis(lims, N):        
        grid_spacing = (lims[1] - lims[0])/N
        hlf = grid_spacing / 2.
        grid_edges = np.linspace(lims[0]-hlf, lims[1]+hlf, N+1)
        return grid_edges, grid_spacing
    
    xe, dx = get_grid_axis(xlims, nx)
    ye, dy = get_grid_axis(ylims, ny)
    ze, dz = get_grid_axis(zlims, nz)
        
    coords, conn, e_center_data = assemble_brute(xe, ye, ze, nx, ny, nz, field, frb_vals)
    return yt.load_unstructured_mesh(conn, coords, elem_data = e_center_data)


def slices_as_unstructured_mesh(slc, field, frb_width=1., resolution=(400,400), **frb_kwargs):
    # to_frb args and kwargs: width, resolution, center=None, height=None, periodic=False   
    frb = slc.to_frb(frb_width, resolution, **frb_kwargs) 
    return unstructured_ds_from_frb_slice(frb, field)
    

In [37]:
import yt
ds = yt.load("IsolatedGalaxy/galaxy0030/galaxy0030")

slc = ds.slice(1, 0.5)
ds_mesh = slices_as_unstructured_mesh(slc, ('gas', 'density'))

yt : [INFO     ] 2021-04-05 16:21:30,608 Parameters: current_time              = 0.0060000200028298
yt : [INFO     ] 2021-04-05 16:21:30,608 Parameters: domain_dimensions         = [32 32 32]
yt : [INFO     ] 2021-04-05 16:21:30,609 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2021-04-05 16:21:30,609 Parameters: domain_right_edge         = [1. 1. 1.]
yt : [INFO     ] 2021-04-05 16:21:30,610 Parameters: cosmological_simulation   = 0
yt : [INFO     ] 2021-04-05 16:21:30,612 Making a fixed resolution buffer of (('gas', 'density')) 400 by 400
Parsing Hierarchy :  99%|█████████▉| 172/173 [00:00<00:00, 13955.86it/s]
yt : [INFO     ] 2021-04-05 16:21:30,633 Gathering a field list (this may take a moment.)


assembling unstructured mesh for slice


yt : [INFO     ] 2021-04-05 16:21:48,176 Parameters: current_time              = 0.0
yt : [INFO     ] 2021-04-05 16:21:48,177 Parameters: domain_dimensions         = [2 2 2]
yt : [INFO     ] 2021-04-05 16:21:48,178 Parameters: domain_left_edge          = [-0.001375   0.4499775 -0.001375 ]
yt : [INFO     ] 2021-04-05 16:21:48,179 Parameters: domain_right_edge         = [1.101375  0.5500275 1.101375 ]
yt : [INFO     ] 2021-04-05 16:21:48,179 Parameters: cosmological_simulation   = 0


unstructured mesh assembled!


In [38]:
import yt_idv
from yt_idv.cameras.trackball_camera import TrackballCamera  # NOQA
from yt_idv.scene_components.mesh import MeshRendering  # NOQA
from yt_idv.scene_data.mesh import MeshData  # NOQA
from yt_idv.scene_graph import SceneGraph  # NOQA


def run_idv(combo_ds):
    rc = yt_idv.render_context(height=800, width=800, gui=True)


    c = TrackballCamera(position=[3.5, 3.5, 3.5], focus=[0.0, 0.0, 0.0])
    rc.scene = SceneGraph(camera=c)

    dd = combo_ds.all_data()
    md = MeshData(data_source=dd)
    md.add_data(("connect1", "density"))
    mr = MeshRendering(data=md)

    rc.scene.data_objects.append(md)
    rc.scene.components.append(mr)
    rc.run()
    
run_idv(combo_ds)

Setting position 560.0 200.0
False False False True


In [41]:
um = ds_mesh.index.meshes[0]


In [43]:
um.connectivity_indices

array([[      0,       1,       2, ...,       5,       6,       7],
       [      8,       9,      10, ...,      13,      14,      15],
       [     16,      17,      18, ...,      21,      22,      23],
       ...,
       [1279976, 1279977, 1279978, ..., 1279981, 1279982, 1279983],
       [1279984, 1279985, 1279986, ..., 1279989, 1279990, 1279991],
       [1279992, 1279993, 1279994, ..., 1279997, 1279998, 1279999]])

In [44]:
dd['connect1','density'].shape

(160000,)

In [45]:
um.connectivity_indices.shape

(160000, 8)

In [55]:
from collections import defaultdict 


def validate_meshes(mesh_ds_list):
    
    flist = mesh_ds_list[0].field_list
    has_fields = True
    for ds in mesh_ds_list:
        has_fields = np.all([fld in flist for fld in ds.field_list])
        if has_fields is False:
            raise ValueError("All 3d mesh-slices must have the same fields to combine!")
    
        
def combine_slices(mesh_ds_list):
    full_coords = []
    full_conn = [ ]
    offset = 0
    full_data = defaultdict(list)
    
    validate_meshes(mesh_ds_list)
    
    for ds in mesh_ds_list:
        ad = ds.all_data()  # all the data in the slice = element centered data
        mesh = ds.index.meshes[0]
        full_coords.append(mesh.connectivity_coords)
        full_conn.append(mesh.connectivity_indices + offset)
        offset = mesh.connectivity_coords.shape[0]
        
        for fld in ds_mesh.field_list:
            if fld[0] != 'all':
                full_data[fld].append(ad[fld])
                
    full_coords = np.concatenate(full_coords)
    full_conn = np.concatenate(full_conn)
    for ky, vals in full_data.items():
        full_data[ky] = np.concatenate(vals)

    return yt.load_unstructured_mesh(full_conn, full_coords, elem_data = full_data)
        

In [56]:
slc = ds.slice(1, 0.5)
ds_mesh = slices_as_unstructured_mesh(slc, ('gas', 'density'))

slc2 = ds.slice(0, 0.5)
ds_mesh2 = slices_as_unstructured_mesh(slc2, ('gas', 'density'))



yt : [INFO     ] 2021-04-05 16:41:19,340 Making a fixed resolution buffer of (('gas', 'density')) 400 by 400


assembling unstructured mesh for slice


yt : [INFO     ] 2021-04-05 16:41:35,891 Parameters: current_time              = 0.0
yt : [INFO     ] 2021-04-05 16:41:35,892 Parameters: domain_dimensions         = [2 2 2]
yt : [INFO     ] 2021-04-05 16:41:35,892 Parameters: domain_left_edge          = [-0.001375   0.4499775 -0.001375 ]
yt : [INFO     ] 2021-04-05 16:41:35,893 Parameters: domain_right_edge         = [1.101375  0.5500275 1.101375 ]
yt : [INFO     ] 2021-04-05 16:41:35,894 Parameters: cosmological_simulation   = 0


unstructured mesh assembled!


yt : [INFO     ] 2021-04-05 16:41:36,128 Making a fixed resolution buffer of (('gas', 'density')) 400 by 400


assembling unstructured mesh for slice


yt : [INFO     ] 2021-04-05 16:41:52,533 Parameters: current_time              = 0.0
yt : [INFO     ] 2021-04-05 16:41:52,534 Parameters: domain_dimensions         = [2 2 2]
yt : [INFO     ] 2021-04-05 16:41:52,534 Parameters: domain_left_edge          = [ 0.4499775 -0.001375  -0.001375 ]
yt : [INFO     ] 2021-04-05 16:41:52,535 Parameters: domain_right_edge         = [0.5500275 1.101375  1.101375 ]
yt : [INFO     ] 2021-04-05 16:41:52,536 Parameters: cosmological_simulation   = 0


unstructured mesh assembled!


In [57]:
combo_ds = combine_slices([ds_mesh, ds_mesh2])

yt : [INFO     ] 2021-04-05 16:41:55,219 Parameters: current_time              = 0.0
yt : [INFO     ] 2021-04-05 16:41:55,219 Parameters: domain_dimensions         = [2 2 2]
yt : [INFO     ] 2021-04-05 16:41:55,220 Parameters: domain_left_edge          = [-0.001375 -0.001375 -0.001375]
yt : [INFO     ] 2021-04-05 16:41:55,221 Parameters: domain_right_edge         = [1.101375 1.101375 1.101375]
yt : [INFO     ] 2021-04-05 16:41:55,221 Parameters: cosmological_simulation   = 0


In [63]:
run_idv(combo_ds)

Setting position 560.0 200.0
False False False True


In [60]:
slc3 = ds.slice(0, 0.4)
ds_mesh3 = slices_as_unstructured_mesh(slc3, ('gas', 'density'))



assembling unstructured mesh for slice


yt : [INFO     ] 2021-04-05 16:46:36,267 Parameters: current_time              = 0.0
yt : [INFO     ] 2021-04-05 16:46:36,267 Parameters: domain_dimensions         = [2 2 2]
yt : [INFO     ] 2021-04-05 16:46:36,268 Parameters: domain_left_edge          = [ 0.3599775 -0.001375  -0.001375 ]
yt : [INFO     ] 2021-04-05 16:46:36,269 Parameters: domain_right_edge         = [0.4400275 1.101375  1.101375 ]
yt : [INFO     ] 2021-04-05 16:46:36,269 Parameters: cosmological_simulation   = 0


unstructured mesh assembled!


In [61]:
combo_ds = combine_slices([ds_mesh, ds_mesh3])

yt : [INFO     ] 2021-04-05 16:46:39,873 Parameters: current_time              = 0.0
yt : [INFO     ] 2021-04-05 16:46:39,874 Parameters: domain_dimensions         = [2 2 2]
yt : [INFO     ] 2021-04-05 16:46:39,874 Parameters: domain_left_edge          = [-0.001375 -0.001375 -0.001375]
yt : [INFO     ] 2021-04-05 16:46:39,875 Parameters: domain_right_edge         = [1.101375 1.101375 1.101375]
yt : [INFO     ] 2021-04-05 16:46:39,876 Parameters: cosmological_simulation   = 0


In [ ]:
run_idv(combo_ds)